In [1]:
import numpy as np
import pandas as pd
import requests
import lxml.html as lh

from tabulate import tabulate
import json, time, datetime
import telegram

In [2]:
# !pip install requests lxml beautifulsoup4 python-telegram-bot tabulate --upgrade

# Global data

In [83]:
def get_data():
    url = 'https://www.worldometers.info/coronavirus'
    page = requests.get(url)
    doc = lh.fromstring(page.content)
    tr_elements = doc.xpath('//*[@id="main_table_countries_today"]/tbody[1]')
    rows=[]
    #For each row, store each first element (header) and an empty list
    for t in tr_elements[0]:
        row = [x.text_content() for x in t.findall('td')[:-1]]
        rows.append(row)
    columns = ['Country', 'Confirmed','New', 'Deaths', 'New Deaths', 'Recovered', 'Active', 'x', 'x']
    df = pd.DataFrame(rows, columns=columns).replace('', None).iloc[:, :-2]
    df = df[['Country', 'Confirmed', 'Recovered', 'Deaths', 'Active']].fillna(0)
    for col in df.columns[1:]:
        df[col] = df[col].astype(str).str.replace(' ','').replace(',','', regex=True).replace('+','').apply(pd.to_numeric)
    df.dropna(inplace=True)
    df['Deaths'] = df['Deaths'].astype(int) 
#     df.loc["Total"] = df.sum()
    return df

df = get_data()
print(len(df))
df.head(300)

100


,Country,Confirmed,Recovered,Deaths,Active
0,China,81171.0,73159.0,3277.0,4735.0
1,Italy,63927.0,7432.0,6077.0,50418.0
2,USA,48778.0,295.0,588.0,47895.0
3,Spain,39676.0,3794.0,2800.0,33082.0
4,Germany,31370.0,749.0,133.0,30488.0
...,...,...,...,...,...
95,Cabo Verde,3.0,2.0,1.0,2.0
96,Sudan,3.0,2.0,1.0,2.0
97,Zimbabwe,3.0,2.0,1.0,2.0
98,Gambia,2.0,2.0,1.0,1.0


# India data

In [3]:
def get_data():
    url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSc_2y5N0I67wDU38DjDh35IZSIS30rQf7_NYZhtYYGU1jJYT6_kDx4YpF-qw0LSlGsBYP8pqM_a1Pd/pubhtml#'
    page = requests.get(url)
    doc = lh.fromstring(page.content)
    tr_elements = doc.xpath('//*[@id="1896310216"]/div/table/tbody')
    rows=[]
    #For each row, store each first element (header) and an empty list
    for t in tr_elements[0]:
        row = [x.text_content() for x in t.findall('td')[:-1]]
        rows.append(row)

    df = pd.DataFrame(rows[1:], columns=rows[0]).replace('', None).dropna().drop_duplicates()
    df[['Confirmed', 'Recovered', 'Deaths', 'Active']] = df[['Confirmed', 'Recovered', 'Deaths', 'Active']].apply(pd.to_numeric)
    return df

df = get_data()
print(len(df))
df.head()

38


,State,,Confirmed,Recovered,Deaths,Active
0,Total,,544,37,9,498
2,Maharashtra,,107,0,2,105
3,Kerala,,97,4,0,93
4,Karnataka,,39,2,1,36
5,Uttar Pradesh,,34,9,0,25


In [9]:
df[['State', 'Confirmed']].iloc[0,1]

517

In [92]:
# df_new = df.copy()
# df_new.Confirmed = list(np.random.randint(50, size=10)) + df.Confirmed.to_list()[10:38]
# df_new = df_new.sample(frac=1)
# df_new.head()

In [51]:
class TelegramMessenger:
    """
    https://forums.fast.ai/t/training-metrics-as-notifications-on-mobile-using-callbacks/17330/4

    Utilizes this API Library:
       https://github.com/python-telegram-bot/python-telegram-bot
    To install:
       pip install python-telegram-bot --upgrade

    {"api_key": "462203107:<your API key>",
     "chat_id": "<your chat ID>"}

    Here's how you get an API key:
       https://core.telegram.org/api/obtaining_api_id
    Here's how you get your chat ID:
       https://stackoverflow.com/questions/32423837/telegram-bot-how-to-get-a-group-chat-id

    """

    def __init__(self, cred_file_path):
        self.__credentials = json.loads(open(cred_file_path).read())
        # Initialize bot
        self.bot = telegram.Bot(token=self.__credentials['api_key'])

    def send_message(self, message='Done'):
        self.bot.send_message(parse_mode='HTML', chat_id=self.__credentials['chat_id'], text=message)

In [94]:
try:
    bot = TelegramMessenger('../india-config.json')
except:
    bot = TelegramMessenger('india-config.json')

In [95]:
curr_date = datetime.datetime.now().date()

while True:
    df_new = get_data()
    df_update = df_new
#     df_update['old_confirmed'] = df.Confirmed
    df_update = df_update.merge(df.rename({'Confirmed': 'old_confirmed'}, axis=1))
    df_update['New'] = df_update['Confirmed'] - df_update['old_confirmed']
    df_update = df_update[df_update['New']>0]
    print(df_update)
    df_update = df_update[['State', 'New', 'Confirmed', 'Deaths']].set_index('State')
    curr_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M")
    curr_time = f'Case update at: {curr_time}'
    print(curr_time)
    if len(df_update)>0:
        message = tabulate(df_update, headers='keys', tablefmt='simple', numalign="right")
        print(message)
        bot.send_message(curr_time)
        bot.send_message(message)
    else:
        print('No new cases')
    date = datetime.datetime.now().date()
    if date != curr_date:
        df = df_new
#     break
    time.sleep(600)

Empty DataFrame
Columns: [State, , Confirmed, Recovered, Deaths, Active, old_confirmed, New]
Index: []
Case update at: 2020-03-23 23:31
No new cases


KeyboardInterrupt: 

In [393]:
# !jupyter nbconvert --to script telegram.ipynb

[NbConvertApp] Converting notebook telegram.ipynb to script
[NbConvertApp] Writing 3206 bytes to telegram.py


In [74]:
# bot = TelegramMessenger('../test-config.json')

In [75]:
# bot.send_message('hi')

In [76]:
# url = f'https://api.telegram.org/bot11352/getUpdates'
# requests.get(url).text